In [1]:
import numpy as np
import math
import copy
import PySimpleGUI as sg
import sys

In [2]:
optimized_flag = False
file_name = "input.txt"
epsilon = 10 ** (-8)

In [3]:
def coord_to_array(a, i, j):
    global optimized_flag
    
    if optimized_flag == False:
        return a[i][j]
    else:
        if j > i:
            aux = i
            i = j
            j = aux
        return a[i*(i+1)//2 + j]

In [4]:
def set_to_array(a, i, j, val):
    global optimized_flag

    if optimized_flag == False:
        a[i][j] = val
        a[j][i] = val
    else:
        if j > i:
            aux = i
            i = j
            j = aux
        a[i*(i+1)//2 + j] = val

In [5]:
def set_to_array_nonsymmetric(a, i, j, val):
    global optimized_flag

    if optimized_flag == False:
        a[i][j] = val
    else:
        if j > i:
            aux = i
            i = j
            j = aux
        a[i*(i+1)//2 + j] = val

In [6]:
def set_to_array_l(a, i, j, val):
    if j > i:
        aux = i
        i = j
        j = aux
    a[i*(i+1)//2 + j] = val

In [7]:
def coord_to_array_l(l, i, j):
    if j > i:
        return 0
    return l[i*(i+1)//2 + j]

In [8]:
def read_input(a, n, p, non_sym):
    global optimized_flag, file_name
    file_read = open(file_name, "r")
    n = int(file_read.readline())
    p = int(file_read.readline())
    
    if non_sym == False and n != p:
        return ([], 1, 2)
    
    if optimized_flag == True:
        for i in range(0, p):
            for j in range(0, n):
                value = float(file_read.readline())
                if i >= j:
                    a = np.append(a, value)
            
    else:
        a = np.zeros((p,n))
        for i in range(0, p):
            for j in range(0, n):
                value = float(file_read.readline())
                if non_sym == False:
                    set_to_array(a, i, j, value)
                else:
                    set_to_array_nonsymmetric(a, i, j, value)
    return (a,n,p)

In [9]:
def initialize_u(u, n, length):
    for i in range(0, n):
        u[i][i] = 1
    return u

In [10]:
def compute_pq(a, n):
    maxim = -1
    for i in range(0, n):
        for j in range(0, i):
            if abs(coord_to_array(a, i,j)) > maxim:
                maxim = abs(coord_to_array(a,i,j))
                p = i
                q = j
    return (p,q)

In [11]:
def compute_tcs(a, n, p, q):
    alfa = (coord_to_array(a,p,p) - coord_to_array(a,q,q))/(2*coord_to_array(a,p,q))
    if alfa >= 0:
        t = -alfa + math.sqrt(alfa*alfa+1)
    else:
        t = -alfa - math.sqrt(alfa*alfa+1)
    
    c = 1 / math.sqrt(1 + t*t)
    s = t / math.sqrt(1 + t*t)
    
    return(t,c,s)

In [12]:
def next_matrix_a(a, n, p, q, t, c, s):    
    for i in range(0, n):
        if i != p and i != q:
#             if p > i:
            aux = coord_to_array(a,p,i)
            set_to_array(a, p, i, c*coord_to_array(a,p,i) + s*coord_to_array(a,q,i))
            set_to_array(a, q, i, -s*aux + c*coord_to_array(a,q,i))

            
    set_to_array(a, p, p, coord_to_array(a,p,p) + t*coord_to_array(a,p,q))
    set_to_array(a, q, q, coord_to_array(a,q,q) - t*coord_to_array(a,p,q))
    set_to_array(a, p, q, 0)
    set_to_array(a, q, p, 0)

    return a

In [13]:
def next_matrix_u(u, n, p, q, t, c, s):
    for i in range(0, n):
        uip_vechi = copy.deepcopy(u[i][p])
        u[i][p] = c * u[i][p] + s * u[i][q]
        u[i][q] = -s * uip_vechi + c * u[i][q]
    return u

In [14]:
def get_error(a_init, n, vp, u):

    error = 0
    au = np.zeros((n,n))
    uvp = np.zeros((n,n))
    
    for i in range(0,n):
        for j in range(0,n):
            value = 0
            for t in range(0,n):
                au[j][i] += coord_to_array(a_init,j,t) * u[t][i]

    for i in range(0, n):
        for j in range(0, n):
            uvp[i][j] = u[i][j] * vp[j]
            
    for i in range(0,n):
        for j in range(0,n):
            error += abs(au[i][j] - uvp[i][j]) * abs(au[i][j] - uvp[i][j])
            
    error = math.sqrt(error)
    return error

In [15]:
def get_valori_proprii():
    global optimized_flag, epsilon
    n = -1
    p1 = -1
    k = 0
    kmax = 10000
    
    a = np.array([])
    (a,n,p1) = read_input(a,n,p1, False)
    print(n, ' ', p1)
    if n != p1:
        return ([], [], [], [], "Nu se poate aplica metoda Jacobi pentru o matrice cu linie != coloana")
    u = np.zeros((n,n))
    a_init = copy.deepcopy(a)
    length = len(a)
    u = initialize_u(u, n, length)
    (p,q) = compute_pq(a,n)
    (t,c,s) = compute_tcs(a,n,p,q)
    print(a)
    while abs(coord_to_array(a,p,q)) > epsilon and k<kmax:
        a = next_matrix_a(a, n, p, q, t, c, s)
        u = next_matrix_u(u, n, p, q, t, c, s)
        (p,q) = compute_pq(a,n)
        (t,c,s) = compute_tcs(a,n,p,q)
                
        k = k+1

#         valori_proprii = np.array([])
#         for i in range(0, n):
#             valori_proprii = np.append(valori_proprii, a[coord_to_array(i,i)])
#         print(valori_proprii)

    valori_proprii = np.array([])
    for i in range(0, n):
        valori_proprii = np.append(valori_proprii, coord_to_array(a,i,i))
    
    get_error(a_init, n, valori_proprii, u)
    
    return (a, a_init, u, valori_proprii, get_error(a_init, n, valori_proprii, u))



In [16]:
def compute_l(a, n):
    global optimized_flag
    
    l = np.array([])
    for i in range(0, int(n*(n+1)/2)):
        l = np.append(l, 0)

                
    for i in range(0, n):
        compute_l_column_p(i, l, a, n)
        
    return l

In [17]:
def compute_l_column_p(column, l, a, n):    
    suma = 0
    for i in range(0, column):
        suma += coord_to_array_l(l,column,i) * coord_to_array_l(l,column,i)
    if coord_to_array(a,column,column) - suma <= 0:
        print('MATRICEA NU ESTE POZITIV DEFINITA!')
        sys.exit()
        

    val = math.sqrt(coord_to_array(a,column,column) - suma)
    set_to_array_l(l, column, column, val)
    
    
    for i in range(column+1, n):
        suma = 0
        for j in range(0, column):
            suma += coord_to_array_l(l,i,j) * coord_to_array_l(l,column, j)
            
        val = coord_to_array(a,i,column) - suma
        val = val / coord_to_array_l(l,column,column)
        set_to_array_l(l, i, column, val)


In [18]:
def product_l(l,n):
    global optimized_flag
    
    if optimized_flag == True:
        new_a = np.array([])
        for i in range(0, int(n*(n+1)/2)):
            new_a = np.append(new_a, 0)
    else:
        new_a = np.zeros((n,n))
                
    for i in range(0,n):
        for j in range(0,i+1):
            suma = 0
            for t in range(0,n):
                suma += coord_to_array_l(l,t,j) * coord_to_array_l(l,t,i)
            set_to_array(new_a, j, i, copy.deepcopy(suma))
                
    return new_a

In [19]:
def cerinta2():
    k = 0
    n = -1
    p = -1
    kmax = 1000
    epsilon = 10 ** (-12)
    a = np.array([])
    (a,n,p) = read_input(a,n,p, False)
    if n != p:
        return "Nu se poate face o descompunere Cholesky pe o matrice cu linie != coloana"
    
    l = compute_l(a, n)
    a_next = product_l(l,n)
    
    while(compare_a(a, a_next, n) > epsilon and k < kmax):
        a = copy.deepcopy(a_next)
        l = compute_l(a, n)
        a_next = product_l(l,n)
        k += 1
        
    vp = np.array([])
    for i in range(0, n):
        vp = np.append(vp, coord_to_array(a_next, i, i))
        
    return vp    

In [20]:
optimized_flag = True

In [20]:
def compare_a(a1, a2, n):
    error = 0
    for i in range(0,n):
        for j in range(0,n):
            error += abs(coord_to_array(a1,i,j) - coord_to_array(a2,i,j))
    
    return math.sqrt(error)

In [21]:
def get_valori_singulare():
    global optimized_flag
    if optimized_flag == True:
        return "Nu se poate calcula cu Numpy pentru o memorare eficienta a matricei"
    n = -1
    p1 = -1
    a = np.array([])
    (a,n,p1) = read_input(a,n,p1, True)
    if n != p1:
        return "Valorile proprii sunt definite pentru matrici patratice"
        
    return np.linalg.eigvals(a)
    
get_valori_singulare()

'Valorile proprii sunt definite pentru matrici patratice'

In [22]:
def get_rang_numpy():
    global optimized_flag
    if optimized_flag == True:
        return "Nu se poate calcula rangul cu Numpy pentru matrice memorata eficient"
    n = -1
    p1 = -1
    a = np.array([])
    (a,n,p1) = read_input(a,n,p1, True)
    return np.linalg.matrix_rank(a)

get_rang_numpy()

3

In [23]:
def get_rang_formula():
    global epsilon, optimized_flag
    n = -1
    p1 = -1
    a = np.array([])
    (a,n,p1) = read_input(a,n,p1, True)
    if optimized_flag == True:
        return "Doar pentru metoda Jacobi se doreste memorare eficienta a matricei"
    if optimized_flag == True and n != p1:
        return "Matricea se doreste a fi memorata eficient dar linie != coloana"
    (u,s,v) = np.linalg.svd(a)
    count = 0
    if n <= p1:
        min_dim = n
    else:
        min_dim = p1
    for i in range(0, min_dim):
        if s[i] > epsilon:
            count += 1
    return count

get_rang_formula()

3

In [24]:
def get_numar_conditionare_numpy():
    global epsilon, optimized_flag
    if optimized_flag == True:
        return "Nu se poate calcula numarul de conditionare cu Numpy pentru matrice memorata eficient"
    n = -1
    p1 = -1
    a = np.array([])

    (a,n,p1) = read_input(a,n,p1, True)
    return np.linalg.cond(a)

get_numar_conditionare_numpy()

54.13783039363558

In [25]:
def get_numar_conditionare_formula():
    global optimized_flag
    if optimized_flag == True:
        return "Nu se poate calcula numarul de conditionare cu Numpy pentru matrice memorata eficient"
    n = -1
    p1 = -1
    min_dim = -1
    a = np.array([])
    (a,n,p1) = read_input(a,n,p1, True)
    (u,s,v) = np.linalg.svd(a)
    
    maxim = -1
    minim = 999999999
    if n <= p1:
        min_dim = n
    else:
        min_dim = p1
    for i in range(0,min_dim):
        if s[i] > maxim:
            maxim = s[i]
        if s[i] < minim and s[i] > epsilon:
            minim = s[i]
            
    return maxim/minim

get_numar_conditionare_formula()

54.13783039363558

In [78]:
def moore_penrose():
    global optimized_flag
    if optimized_flag == True:
        return "P >= N, deci nu poate fi memorata matricea eficient"
    epsilon = 10 ** (-8)
    n = -1
    p1 = -1
    a = np.array([])
    (a,n,p1) = read_input(a,n,p1, True)
    (u, s, v) = np.linalg.svd(a)
    
    if p1 <= n:
        coord_min = p1
    else:
        coord_min = n
    
    for i in range(0, coord_min):
        if abs(s[i]) > epsilon:
            s[i] = 1 / s[i]
            
    sinv = np.zeros((n,p1))
    
    for i in range(0, coord_min):
        sinv[i][i] = s[i]
    
    return v.T.dot(sinv).dot(u.T)

moore_penrose()

array([[ 0.72701733, -0.62234848,  0.22277247,  0.56293906],
       [-0.44874681,  0.90442695, -0.41465283, -0.45964206],
       [ 0.15103463, -0.41175417,  0.23829837,  0.18053948]])

In [79]:
def moore_penrose2():
    global optimized_flag
    if optimized_flag == True:
        return "P >= N, deci nu poate fi memorata matricea eficient"
    n = -1
    p1 = -1
    a = np.array([])
    (a,n,p1) = read_input(a,n,p1, True)
    
    result = copy.deepcopy(np.transpose(a))
    result = np.dot(result, a)
    result = np.linalg.inv(result)
    result = np.dot(result, np.transpose(a))
    return result

moore_penrose2()

array([[ 0.72701733, -0.62234848,  0.22277247,  0.56293906],
       [-0.44874681,  0.90442695, -0.41465283, -0.45964206],
       [ 0.15103463, -0.41175417,  0.23829837,  0.18053948]])

In [28]:
def compare_inverses():
    global optimized_flag
    if optimized_flag == True:
        return "P >= N, deci nu poate fi memorata matricea eficient"
    n = -1
    p1 = -1
    a = np.array([])
    (a,n,p1) = read_input(a,n,p1, True)
    matrix1 = moore_penrose()
    matrix2 = moore_penrose2()
    error = 0
    
    for i in range(0, n):
        for j in range(0, n):
            error += abs(matrix1[i][j] - matrix2[i][j])
    
    return math.sqrt(error)

compare_inverses()

2.0390647923058285e-07

In [80]:
def draw_interface():
    global l, window, optimized_flag, epsilon
 

    layout = [[sg.Text("Introduceti epsilon:")], 
              [sg.In(size=(25, 1), enable_events=True, key="EPS")],
              [sg.Button("GATA")],
              
              [sg.Button("Valori proprii cu metoda Jacobi")],
              [sg.Text(size=(100, 1), key="VPJACOBI")],
              
              [sg.Button("Vectori proprii cu metoda Jacobi")],
              
              [sg.Button("||A_init U − UΛ||.")],
              [sg.Text(size=(100, 1), key="EroareJACOBI")],

              [sg.Button("Valori proprii Cholesky")],
              [sg.Text(size=(100, 1), key="VPCHOLESKY")],

              [sg.Button("Valori proprii Numpy")],
              [sg.Text(size=(100, 1), key="VPNUMPY")],
              
              [sg.Button("Rang formula")],
              [sg.Text(size=(100, 1), key="RF")],
        
              [sg.Button("Rang Numpy")],
              [sg.Text(size=(100, 1), key="RN")],
        
              [sg.Button("Numar conditionare formula")],
              [sg.Text(size=(100, 1), key="NCF")],
        
              [sg.Button("Numar conditionare Numpy")],
              [sg.Text(size=(100, 1), key="NCN")],
              
              [sg.Button("Eroare pseudoinversa Moore-Penrose")],
              [sg.Text(size=(100, 1), key="MP")],

              [sg.Button("EXIT")]
             ]
    window = sg.Window("Calcul numeric - Laborator 5", layout)

    while True:
        event, values = window.read()

        if event == "EXIT":
            break
        elif event == "Valori proprii cu metoda Jacobi":
            (a, a_init, u, vp, error) = get_valori_proprii()
            if a == []:
                window["VPJACOBI"].update(error)
            else:
                window["VPJACOBI"].update(vp)
        elif event == "Vectori proprii cu metoda Jacobi":
            (a, a_init, u, vp, error) = get_valori_proprii()
            print(u)
        elif event == "||A_init U − UΛ||.":
            (a, a_init, u, vp, error) = get_valori_proprii()
            window["EroareJACOBI"].update(error)
        elif event == "Valori proprii Cholesky":
            window["VPCHOLESKY"].update(cerinta2())
        elif event == "Valori proprii Numpy":
            print(get_valori_singulare())
            window["VPNUMPY"].update(get_valori_singulare())
        elif event == "Rang formula":
            window["RF"].update(get_rang_formula())
        elif event == "Rang Numpy":
            window["RN"].update(get_rang_numpy())
        elif event == "Numar conditionare formula":
            window["NCF"].update(get_numar_conditionare_formula())
        elif event == "Numar conditionare Numpy":
            window["NCN"].update(get_numar_conditionare_numpy())
        elif event == "Eroare pseudoinversa Moore-Penrose":
            window["MP"].update(compare_inverses())
        elif event == "GATA":
            epsilon = float(values["EPS"])

    window.close()
    
draw_interface()

1   2


In [77]:
file_name = "input.txt"
optimized_flag = False

In [253]:
file_name = "input.txt"
optimized_flag = True

In [75]:
file_name = "input2.txt"
optimized_flag = False

In [36]:
file_name = "input2.txt"
optimized_flag = True

In [46]:
file_name = "input3.txt"
optimized_flag = False

In [50]:
file_name = "input4.txt"
optimized_flag = False

In [71]:
file_name = "input5.txt"
optimized_flag = True

In [73]:
file_name = "input5.txt"
optimized_flag = False